In [61]:
import findspark
#初始化
findspark.init()
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession



In [62]:
# 定义数据库的地址，以及表，登录用户及密码
url = "jdbc:mysql://localhost:3306/sxzy"
table="gas_bank_records_2016"
#密码账户需要字典的形式传入
properties ={"user":"root","password":"521521"}
spark = SparkSession.builder.appName('My first app').getOrCreate()
df = spark.read.jdbc(url=url,table=table,properties=properties)
df.show()


+-------------------+-------------------+------+-------------------+-------------------+----+--------+----------+--------------------+--------------------------------------+----------------------------+--------------------------------+-------------------------------------+---+----+----+
|               cxzh|               cxkh|  jymc|               jyrq|               jysj|jdbz|    jyje|      jyye|             jydfzkh|                                jydfmc|                   jydfzhkhh|                            zysm|                                 beiz| pc|  ip| mac|
+-------------------+-------------------+------+-------------------+-------------------+----+--------+----------+--------------------+--------------------------------------+----------------------------+--------------------------------+-------------------------------------+---+----+----+
|   6226093580025203|   6226093580025203|乔秀秀|2015-03-17 12:50:31|2015-03-17 12:50:31|  进| 58000.0|  58360.66|    6217685500126432|      

In [63]:

df.show(5)


+-------------------+-------------------+------+-------------------+-------------------+----+-------+--------+----------------+------+----------------------------+------------------+-------------------------------------+---+----+----+
|               cxzh|               cxkh|  jymc|               jyrq|               jysj|jdbz|   jyje|    jyye|         jydfzkh|jydfmc|                   jydfzhkhh|              zysm|                                 beiz| pc|  ip| mac|
+-------------------+-------------------+------+-------------------+-------------------+----+-------+--------+----------------+------+----------------------------+------------------+-------------------------------------+---+----+----+
|   6226093580025203|   6226093580025203|乔秀秀|2015-03-17 12:50:31|2015-03-17 12:50:31|  进|58000.0|58360.66|6217685500126432|任雅文|中信银行股份有限公司吕梁支行|              null|                                 null|1.0|null|null|
|   6226193900031475|   6226193900031475|武利军|2013-12-20 15:24:16|2013-12-20 15:24

In [31]:
df  # 查看字段，通过字段回去需要的数据

DataFrame[cxzh: string, cxkh: string, jymc: string, jyrq: string, jysj: string, jdbz: string, jyje: string, jyye: string, jydfzkh: string, jydfmc: string, jydfzhkhh: string, zysm: string, beiz: string, pc: string, ip: string, mac: string]

In [7]:
df.count(),len(df.columns)

(57347, 16)

In [8]:
df.printSchema()# 元数据分析，查看字段类型

root
 |-- cxzh: string (nullable = true)
 |-- cxkh: string (nullable = true)
 |-- jymc: string (nullable = true)
 |-- jyrq: string (nullable = true)
 |-- jysj: string (nullable = true)
 |-- jdbz: string (nullable = true)
 |-- jyje: string (nullable = true)
 |-- jyye: string (nullable = true)
 |-- jydfzkh: string (nullable = true)
 |-- jydfmc: string (nullable = true)
 |-- jydfzhkhh: string (nullable = true)
 |-- zysm: string (nullable = true)
 |-- beiz: string (nullable = true)
 |-- pc: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- mac: string (nullable = true)



In [19]:
df1=df.select(['cxkh','cxzh']) # 按照列名选择

In [26]:
df2=df.filter((df['jyje']>100000) & (df['jysj']>'2015-01-01')) #按照条件进行筛选

In [45]:
df3=df2['jyje','jyye']

In [32]:
df2['cxzh','jymc'].show(5,False)

+----------------+------+
|cxzh            |jymc  |
+----------------+------+
|6217680703338579|王涛  |
|6217680703338579|王涛  |
|6217685500184324|郑婷婷|
|6226093580025203|乔秀秀|
|4720683943556666|陈月娥|
+----------------+------+
only showing top 5 rows



In [33]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [48]:
# 类型转换
df3 = df3.withColumn("jyje",df3.jyje.astype("int"))
df3 = df3.withColumn("jyye",df3.jyye.astype("int"))

In [49]:
# 生成密集性向量
assembler = VectorAssembler(inputCols=df3.columns,outputCol="features")
df_new=assembler.transform(df3)

In [52]:
df_new.show()

+-------+-------+--------------------+
|   jyje|   jyye|            features|
+-------+-------+--------------------+
| 350000| 350213| [350000.0,350213.0]|
| 120000| 121213| [120000.0,121213.0]|
|1000000|1034572|[1000000.0,103457...|
| 234000| 300510| [234000.0,300510.0]|
| 500000| 500000| [500000.0,500000.0]|
|1004986|1004987|[1004986.0,100498...|
| 200000| 328767| [200000.0,328767.0]|
| 110000| 202807| [110000.0,202807.0]|
|1005130|1005154|[1005130.0,100515...|
| 114300| 119454| [114300.0,119454.0]|
| 500000| 570807| [500000.0,570807.0]|
|1005178|1005232|[1005178.0,100523...|
| 250000| 250000| [250000.0,250000.0]|
| 500000| 500000| [500000.0,500000.0]|
|1000000|1000000|[1000000.0,100000...|
| 120000| 120000| [120000.0,120000.0]|
| 502732| 502733| [502732.0,502733.0]|
| 301639| 301640| [301639.0,301640.0]|
|1000000|1000971|[1000000.0,100097...|
|1000000|2000971|[1000000.0,200097...|
+-------+-------+--------------------+
only showing top 20 rows



In [58]:
Correlation.corr(df_new,'features').show()## pearson相关性

+--------------------+
|   pearson(features)|
+--------------------+
|1.0              ...|
+--------------------+



In [60]:
Correlation.corr(df_new,'features', 'spearman').show()# 计算spearman系数

+--------------------+
|  spearman(features)|
+--------------------+
|1.0              ...|
+--------------------+



In [64]:
df.describe().show()

+-------+--------------------+--------------------+-----+--------------------+-------------------+-----+------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------------------+------------------+-----------------+--------------------+
|summary|                cxzh|                cxkh| jymc|                jyrq|               jysj| jdbz|              jyje|              jyye|             jydfzkh|              jydfmc|          jydfzhkhh|                zysm|                            beiz|                pc|               ip|                 mac|
+-------+--------------------+--------------------+-----+--------------------+-------------------+-----+------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------------------+------------------+-----------------+--------------------+
|  count|               57347|               5734

In [66]:
df3.describe().show()  # 描述性统计

+-------+-----------------+-----------------+
|summary|             jyje|             jyye|
+-------+-----------------+-----------------+
|  count|             3867|             3867|
|   mean| 925364.859839669|1224231.370312904|
| stddev|1726854.324281844|2864617.563459842|
|    min|           100107|         -1003514|
|    max|         20000000|         32905305|
+-------+-----------------+-----------------+



In [67]:
df3.printSchema()#查看字段属性

root
 |-- jyje: integer (nullable = true)
 |-- jyye: integer (nullable = true)



In [72]:
df.filter(df['jymc']=='崔新平').show(2)  # 样本选择

+----------------+----------------+------+-------------------+-------------------+----+--------+---------+-------+------+---------+--------+----+---+----+----+
|            cxzh|            cxkh|  jymc|               jyrq|               jysj|jdbz|    jyje|     jyye|jydfzkh|jydfmc|jydfzhkhh|    zysm|beiz| pc|  ip| mac|
+----------------+----------------+------+-------------------+-------------------+----+--------+---------+-------+------+---------+--------+----+---+----+----+
|5240470001156763|5240470001156763|崔新平|2012-05-31 11:39:40|2012-05-31 11:39:40|  进|100000.0|100818.43|   null|  null|     null|    转款|null|4.0|null|null|
|5240470001156763|5240470001156763|崔新平|2012-04-01 23:59:59|2012-04-01 23:59:59|  出|    16.5|   818.43|   null|  null|     null|透支利息|null|4.0|null|null|
+----------------+----------------+------+-------------------+-------------------+----+--------+---------+-------+------+---------+--------+----+---+----+----+
only showing top 2 rows



In [71]:
df.filter(df['jymc']=='崔新平').select('cxkh','jdbz','jyje').show(2) # 样本+变量选择

+----------------+----+--------+
|            cxkh|jdbz|    jyje|
+----------------+----+--------+
|5240470001156763|  进|100000.0|
|5240470001156763|  出|    16.5|
+----------------+----+--------+
only showing top 2 rows



In [74]:
df.withColumn('test',(df['jyje']+100)).select('jyje','test').show(5) # 增加字段

+-------+-------+
|   jyje|   test|
+-------+-------+
|58000.0|58100.0|
|   10.0|  110.0|
| 1900.0| 2000.0|
|  19.75| 119.75|
|   1.58| 101.58|
+-------+-------+
only showing top 5 rows



In [75]:
df.fillna('wz')

DataFrame[cxzh: string, cxkh: string, jymc: string, jyrq: string, jysj: string, jdbz: string, jyje: string, jyye: string, jydfzkh: string, jydfmc: string, jydfzhkhh: string, zysm: string, beiz: string, pc: string, ip: string, mac: string]

In [77]:
df.groupBy('jymc').count().show()

+------------------------------+-----+
|                          jymc|count|
+------------------------------+-----+
|                        张丽丽|  103|
|                        高晓兰|  132|
|                        熊嵩嵩|  107|
|                        郑沁彬|  326|
|                        高明明|  197|
|                        张艳文|  600|
|                        吴晓虎| 1640|
|                        吴春旺|   36|
|                        李娇娇| 1134|
|                        李明则|  550|
|                        乔秀秀| 3401|
|                          官侠|   70|
|                          刘涛|  701|
|吕梁市建筑安装总公司第四工程处| 5608|
|                        王建伟| 1601|
|                          严斌|   59|
|                          null|  341|
|                          曾超| 2445|
|                        鲁立国|  292|
|                        芦继香|   54|
+------------------------------+-----+
only showing top 20 rows



In [91]:
# df.select('zysm').show()
ddd=df[df['zysm'].isNull()]

In [94]:
ddd1=ddd['cxkh','zysm']

In [98]:
ddd1.fillna('wz')

DataFrame[cxkh: string, zysm: string]

In [99]:
ddd1.show()

+-------------------+----+
|               cxkh|zysm|
+-------------------+----+
|   6226093580025203|null|
|   6226093580019743|null|
|6216618110000830445|null|
|   6226093580019743|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6217680703338579|null|
|   6226093580019743|null|
|   6226093580019743|null|
|   6226093580019743|null|
|   6226093580019743|null|
|   4100623582007688|null|
|   4100623582007688|null|
|   4100623582007688|null|
+-------------------+----+
only showing top 20 rows

